# Process RR_peaks to metrics

**The following notebook** 
* Reads the RR peaks file
* Derives metrics from 5 min intervals
* Joins clinical indicator file with metrics on patient id 
* Saves the file

The original data comes from 

In [8]:
import pandas as pd
import h5py
import project_utils.rr_to_metrics as helper

read_path = "processed_data/Nabian_2018_derrived_data.h5"
pickle_path = 'processed_data/10min_Nabian.pkl'

Read the RR_Peaks file ["our_processing\ML_Data.h5"]("our_processing\ML_Data.h5")

In [9]:
def read_data(file_path, group_name, signal_name= "r_intervals"):
    with h5py.File(file_path, "r") as hdf:
        group = hdf[group_name]
        data = group[signal_name][()]
    return data

def get_page_names(file_path):
    with h5py.File(file_path, "r") as hdf:
        page_names = list(hdf.keys())
    return page_names

page_names = get_page_names(read_path)

patient_arrays = {group_name: read_data(read_path, group_name) for group_name in page_names}

print(patient_arrays)

{'19070921': array([764., 772., 780., ..., 708., 696., 704.]), '19072205': array([824., 772., 776., ..., 820., 800., 804.]), '19072214': array([1268.,  632.,  644., ..., 1332., 1576., 1812.]), '19072938': array([684., 700., 724., ..., 688., 676., 448.]), '19072939': array([732., 720., 668., ..., 656., 664., 556.]), '19072940': array([ 668.,  656.,  656., ..., 1016.,  472.,  912.]), '19080106': array([892., 888., 856., ..., 816., 816., 560.]), '19080715': array([1100.,  948.,  964., ..., 1228., 1044., 1440.]), '19081506': array([ 884.,  832.,  824., ..., 1016.,  520., 1608.]), '19082406': array([ 692.,  680.,  668., ..., 1780.,  792., 1276.]), '19090308': array([624., 624., 620., ..., 764., 768., 748.]), '19090320': array([708., 712., 684., ..., 636., 644., 620.]), '19101607': array([1508.,  956.,  972., ...,  604.,  848., 1388.]), '19101619': array([1356.,  952.,  944., ...,  740.,  752.,  748.]), '19102102': array([784., 804., 784., ..., 648., 648., 644.]), '19102103': array([ 744.,  

Calculate the metrics for each patient.

In [10]:
metrics_df = pd.DataFrame()

for id, signal in patient_arrays.items():
    individual_metrics = helper.patients_metrics(signal, sub_signal_duration_s=600)
    individual_metrics['id'] = id
    print(id, "processed")
    metrics_df = pd.concat([metrics_df, individual_metrics], ignore_index=True)
    

19070921 processed


c:\GITHUB\HRV\HRV-EDA\metrics\HRV_Metrics.py:127: RuntimeWarning: divide by zero encountered in scalar divide
  return self.LF_power() / self.HF_power()


19072205 processed
19072214 processed
19072938 processed
19072939 processed
19072940 processed
19080106 processed
19080715 processed
19081506 processed
19082406 processed
19090308 processed
19090320 processed
19101607 processed
19101619 processed
19102102 processed
19102103 processed
19102524 processed


c:\GITHUB\HRV\HRV-EDA\metrics\HRV_Metrics.py:127: RuntimeWarning: divide by zero encountered in scalar divide
  return self.LF_power() / self.HF_power()


19102622 processed
19112609 processed
19120302 processed
19120323 processed
19120704 processed
19120723 processed
19121303 processed
19121735 processed
20010826 processed
20010827 processed
20011712 processed
20050628 processed
20052606 processed
20061729 processed
20092226 processed
20092535 processed
20101424 processed
20101822 processed
20102029 processed
20120116 processed
20120922 processed
20121033 processed
20121716 processed
20121718 processed
20122932 processed
20123017 processed


Read the clinical indicators

In [11]:
clin_indic = helper.df_from_excel('actionable_data/Clinical indicators.xlsx')
clin_indic.rename(columns={'number': 'id'}, inplace=True)
clin_indic['id'] = clin_indic['id'].astype(str)
display(clin_indic.columns)

Index(['id', 'gender', 'age', 'height', 'weight', 'admission FBG (mmol/L)',
       'Discharge FBG (mmol/L)', 'HbA1c (%)', 'SBP (mmHg)', 'DBP (mmHg)',
       'WBC (×109/L)', 'N% (%)', 'Hb (g/L)', 'PLT (×109/L)', 'CRP (mg/L)',
       'ALT (U/L)', ' AST (U/L)', 'AST/ALT', 'GGT (U/L)', 'BUN (mmol/L)',
       ' UA (mmol/L)', 'TG (mmol/L)', 'HDL-C (mmol/L)', 'LDL-C (mmol/L)',
       'UMA (mg)', 'UCr (g)', 'UACR (mg/g)', 'Diabetic Complications',
       'Diabetic nephropathy', 'Diabetic retinopathy and cataract',
       'Diabetic peripheral neuropathy',
       'Coronary artery disease and cardiac insufficiency',
       'Lower extremity atherosclerosis or stenosis', 'Carotid plaque'],
      dtype='object')

Inner merge the clinical indicators and processing results

In [12]:
metrics_df["id"].astype(str)
merged_data = metrics_df.merge(clin_indic, on='id', how='inner')

merged_data.head()

,t_start,t_end,SDRR,RMSSD,pNN50 (%),Mean HR (bpm),ULF Power,ULF Peak Frequency,VLF Power,VLF Peak Frequency,...,UMA (mg),UCr (g),UACR (mg/g),Diabetic Complications,Diabetic nephropathy,Diabetic retinopathy and cataract,Diabetic peripheral neuropathy,Coronary artery disease and cardiac insufficiency,Lower extremity atherosclerosis or stenosis,Carotid plaque
0,0.764,600.396,59.953222,50.339557,4.161567,81.746048,0.004173,0.000000,3.314667,0.030018,...,411.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,601.064,1200.808,92.834423,81.165262,12.719892,73.949222,0.036403,0.001667,10.496690,0.030013,...,411.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1201.652,1801.208,76.874801,72.754984,13.532764,70.253165,0.002963,0.001668,4.764365,0.026686,...,411.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1801.996,2401.520,65.979059,54.859842,10.431154,71.962579,0.015285,0.000000,5.192408,0.038364,...,411.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2402.412,3002.176,56.022949,32.686929,3.585147,78.114595,0.014553,0.000000,11.302079,0.035014,...,411.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
merged_data.to_pickle(pickle_path)
merged_data.to_csv(pickle_path.replace(".pkl", '.csv'), index=False)